# Практическая работа №2 Основы Python

## Задача 1

Необходимо спарсить данные о вакансиях python разработчиков с сайта hh.ru, введя в поиск “python разработчик” и указав,
что мы рассматриваем все регионы. Необходимо спарсить:
*Название вакансии
*Требуемый опыт работы
*Заработную плату
*Регион

И сохранить эти данные в формате json в следующем виде:
<img src="https://lhd8ts4trw.a.trbcdn.net/cdn/yusuciqe45/00_data_engineer/pic/1_3_question.png" style="width: 40%">

### Импорт библиотек

In [153]:
import json

import requests
import time
from bs4 import BeautifulSoup

### Функции

In [154]:
def get_url(experience, vacancy, on_page, page):
    return f'https://hh.ru/search/vacancy?experience={experience}&search_field=name&search_field=company_name&search_field=description&text={vacancy}&no_magic=true&L_save_area=true&items_on_page={on_page}&page={page}&hhtmFrom=vacancy_search_list'


def clean_html(string):
    if string:
        string = str(string)
        if 'div' in string:
            string = string.split('>')[1].split('<')[0]
        elif 'span' in string:
            string = string.split('">')[1].split('</')[0]
            string = string.replace('<!-- -->', '').replace('  ', ' ').replace('\u202f', ' ')
        else:
            string = string.split('>')[1].split('<')[0]
        return string

### Переменные

In [166]:
on_page = '20'
vacancy = 'python+разработчик'
experiences = [
    'between3And6',
    'between1And3',
    'noExperience',
    'moreThan6'
]
experiences_for_dict = [
    'От 3 до 6 лет',
    'От 1 года до 3 лет',
    'Нет опыта',
    'Более 6 лет'
]
url_main_search_page = 'https://hh.ru/search/vacancy?no_magic=true&L_save_area=true&text=python+разработчик&salary=&currency_code=RUR&experience=doesNotMatter&order_by=relevance&search_period=0&items_on_page=20'

### Определяем количество вакансий в разрезе по опыту

In [167]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'
}

main_search_page = requests.get(url_main_search_page, headers=headers, timeout=5)

In [168]:
soup_main_search_page = BeautifulSoup(main_search_page.text, 'lxml')

In [169]:
list_of_params = soup_main_search_page.find_all(
    "span"
    , {
        "class"
        :
            "bloko-text bloko-text_tertiary"
    })

In [170]:
count_of_pages = []
for n, i in enumerate(list_of_params[21:25]):
    i = str(i).replace('\u202f', '')
    x = i.split('>')[1].split('<')[0]
    count_of_pages.append(int(x))
count_of_pages

[2276, 1874, 612, 214]

### Делаем обход страниц по заданным параметрам поиска

In [176]:
dict_to_write = {'data': []}
for n, experience in enumerate(experiences):
    for page in range(count_of_pages[n] // int(on_page) + 1):
        target_page = get_url(experience, vacancy, on_page, page)
        html_target_page = requests.get(target_page, headers=headers, timeout=5)
        time.sleep(3)
        soup = BeautifulSoup(html_target_page.text, 'lxml')
        for link in soup.find_all('div', class_='vacancy-serp-item-body'):
            title = link.find("a", {"data-qa": "serp-item__title"}).text
            salary = clean_html(link.find("span", {"data-qa": "vacancy-serp__vacancy-compensation"}))
            region = clean_html(link.find("div", {"data-qa": "vacancy-serp__vacancy-address"}))
            dict_to_write['data'].append({
                'title': title,
                'work experience': experiences_for_dict[n],
                'salary': salary,
                'region': region
            })
            with open(f'{n}_data_{experience}.json', 'w', encoding='utf-8') as file:
                json.dump(dict_to_write, file, ensure_ascii=False)
    time.sleep(30)
    print(len(dict_to_write['data']))
    dict_to_write = {'data': []}

2000


## Задача 2

**Палиндром строки**

Дана строка X, возвращайте True, если X является палиндромом.

Строка является палиндромом тогда, когда она читается одинаково как в обратном, так и в прямом направлении.

Например, является “taco cat” является палиндромом, в то время как “black cat” не является.
В данной задаче пробелы не учитываются.

Гарантируется, что исходная строка может содержать символы только нижнего регистра.

In [177]:
def check_poly(x):
    x = x.replace(' ', '')
    return x == x[::-1]

In [178]:
check_poly('taco cat')

True

In [179]:
check_poly('tacdo cat')

False

## Задача 3

**Перевод арабского числа в римское**

Дано целое положительное число X, необходимо вывести вариант этого числа в римской системе счисления в формате строки.

<img src="https://lhd8ts4trw.a.trbcdn.net/cdn/yusuciqe45/00_data_engineer/pic/1_3_table.png" style="width: 40%">

Римские числа записываются от наибольшего числа к наименьшему слева направо.

Однако число 4 не является “IIII”. Вместо этого число 4 записывается, как “IV”. Т.к. 1 стоит перед 5, мы вычитаем 1, делая 4. Тот же принцип применим к числу 9, которое записывается как “IX”.

Случаи, когда используется вычитание:

I может быть помещен перед V и X, чтобы сделать 4 и 9.

X может быть помещен перед L и C, чтобы составить 40 и 90.

C может быть помещен перед D и M, чтобы составить 400 и 900.



In [180]:
def int_to_roman(num):
    def add_to_result(res, x, n):
        x: int = int(x)
        if x == 9:
            res = res + itr_2[9 * 10 ** n]
        elif x == 4:
            res = res + itr_2[4 * 10 ** n]
        else:
            f: int = x // 5
            e: int = x - 5 * f
            res = res + itr[5 * 10 ** n] * f + itr[1 * 10 ** n] * e
        return res

    itr = {
        1: 'I',
        5: 'V',
        10: 'X',
        50: 'L',
        100: 'C',
        500: 'D',
        1000: 'M',
    }
    itr_2 = {
        4: 'IV',
        9: 'IX',
        40: 'XL',
        90: 'XC',
        400: 'CD',
        900: 'CM',
    }
    result: str = ''
    string_num: str = str(num)
    if len(string_num) == 4:
        result = itr[1000] * int(string_num[0])
        string_num = string_num[1:]
    if len(string_num) in [3, 4]:
        result = add_to_result(result, string_num[0], 2)
        result = add_to_result(result, string_num[1], 1)
        result = add_to_result(result, string_num[2], 0)
    if len(string_num) == 2:
        result = add_to_result(result, string_num[0], 1)
        result = add_to_result(result, string_num[1], 0)
    if len(string_num) == 1:
        result = add_to_result(result, string_num[0], 0)
    return result

In [181]:
int_to_roman(174)

'CLXXIV'

## Задача 4

**Валидность скобок**

Дана строка X, состоящая только из символов “{“, “}”, “[“, “]”, “(“, “)”. Программа должна вывести True, в том случае если все открытые скобки закрыты. Например: “[()]{}”, все открытые скобки закрыты закрывающимися скобками, потому вывод будет True. В случае же, если строка будет похожа на: “{{{}”, то вывод будет False, т.к. не все открытые скобки закрыты.

In [182]:
def close_bracket(x):
    while True:
        len_x = len(x)
        x = x.replace('()', '').replace('{}', '').replace('[]', '')
        if len_x == len(x):
            break
    if len_x != 0:
        return False
    return True

In [183]:
close_bracket('[()]{}')

True

In [184]:
close_bracket('[({)]{}')

False

In [185]:
close_bracket('}')

False

## Задача 5

**Умножить два бинарных числа в формате строк**

На вход подаются две строки X1 и X2, содержащие бинарные числа.

Необходимо вывести их бинарное произведение в формате строки.

In [188]:
def multiply_binary(a, b):
    if a == '0' and b == "0":
        return '0'
    return str(bin(int(a, 2)*int(b, 2))).lstrip('0b')

In [189]:
multiply_binary(a = "1010", b = "1011")

'1101110'

In [190]:
multiply_binary(a = "0", b = "0")

'0'

In [191]:
multiply_binary(a = "111", b = "101")

'100011'